<a href="https://colab.research.google.com/github/josephmailil1/Speciale---Stibo/blob/main/LLM_Flow_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

In [2]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_Key') #use the name of your colab key

In [3]:
import pandas as pd

shap = pd.read_csv("shap.csv")

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, openai_api_key=openai_api_key)

In [5]:
pip install ucimlrepo

In [6]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features # Importing all features
y = adult.data.targets # Importing target varialbes

In [7]:
# Recoding to match the same
y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})

<ipython-input-7-65ecf31535d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})


In [8]:
data =  pd.concat([X, y], axis=1)

In [9]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

tool = PythonAstREPLTool(locals={"data": data, "shap": shap})
tool.invoke("data['age'].mean()")

38.64358543876172

In [12]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
llm_with_tools.invoke(
    "I have a dataframe 'data' and want to know the correlation between the 'age' and 'capital-gain' columns"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p6A5TpMdxKjWd8ipqLW6s0zg', 'function': {'arguments': '{"query":"import pandas as pd\\ndata = pd.DataFrame({\'age\': [25, 30, 35, 40, 45], \'capital-gain\': [5000, 15000, 25000, 35000, 45000]})\\ndata[[\'age\', \'capital-gain\']].corr()"}', 'name': 'python_repl_ast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 126, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_76f018034d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fbcb43c1-0f77-4b60-b1df-9bcdb7f98f79-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "import pandas as pd\ndata = pd.DataFrame({'age': [25, 30, 35, 40, 45], 'capital-gain': [5000, 15000, 25000, 35000, 45000]})\ndata[['age', 'capital-gain']].corr()"}, 'id': 'call_p6A5TpMdxKjWd8ipqLW6s0zg'}])

In [13]:
from langchain.output_parsers import JsonOutputToolsParser

In [14]:
parser = JsonOutputToolsParser(tool = tool)

In [19]:
system = f"""You have access to a pandas dataframe `data`. \
Here is the output of `data.head().to_markdown()`:

```
{data.head().to_markdown()}
```
You are also given the pandas dateframe shap. Containing shap values for the test set obsevations.
The shap obsevation shows which values are important for the final prediction
```
{shap.head().to_markdown()}
```
Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

In [35]:
chain = prompt | llm_with_tools | parser  # noqa
chain.invoke({"question": "What is the most important feature value for the first obsevation?'"})

[{'args': {'query': 'data.iloc[0]'}, 'type': 'python_repl_ast'}]

In [21]:
data['age'].corr(data['capital-gain'])

0.07722902236438102

In [62]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe `data`. \
Here is the output of `data.head().to_markdown()`:

```
{data.head().to_markdown()}
```

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas.
Respond directly to the question once you have enough information to answer it."""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser )
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["response"])
)

In [63]:
from langchain.memory import ConversationBufferMemory
# This is do give the bot memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [77]:
test = chain.invoke({"question": "What is the distribution of sex in the data'"})
test

{'response': 'The distribution of sex in the data is as follows:\n\n- Male: 3\n- Female: 1\n\nThis indicates there are three males and one female in the sample provided.'}

In [78]:
def call_chain(user_question):
    response = chain.invoke({"question": user_question})
    return response

# Interface - Streamlit

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.1 MB/s eta 0:00:00


In [ ]:
import streamlit as st

In [ ]:
st.title("Ask the Document")
if "messages" not in st.session_state.keys():
    st.session_state.messages = [
        {"role": "assistant", "content": "Ask me a question !"}
    ]

chat_engine = prompt.as_chat_engine(chat_mode="condense_question", verbose=True)

if prompt := st.chat_input("Your question"):
    st.session_state.messages.append({"role": "user", "content": prompt})

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = chat_engine.chat(prompt)
            st.write(response.response)
            pprint_response(response, show_source=True)
            message = {"role": "assistant", "content": response.response}
            st.session_state.messages.append(message)


TypeError: 'RunnableSequence' object is not callable

# Interface - Gradio

## Adding advanced options

You wanna get nuts? Let's get nuts.

We add the following:

* Title
* Description
* Three examples
* Slider to change the temperature
* Avatar images for the bot and the user

#new

In [30]:
pip install gradio

In [48]:
import gradio as gr

In [72]:
demo = gr.Interface(
    fn=call_chain,
    inputs=["text"],
    outputs=["text"]
)

In [73]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://138fd0df42af87a154.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
